**Before running the code, scroll to the bottom and select the map that you want the map HTML to save as, otherwise it will just save as map.html**

In [6]:
from pyproj import Transformer
import folium
import pandas as pd

In [7]:
#Upload the csv from the Street Manager download
Reinstatements_Data =pd.read_csv("Reinstatements.csv") 
DD = Reinstatements_Data #This is just here so that the code can be changed to use with other datasets without having to change the reference throughout the code
import re

In [9]:
#These are built in so that the code can be reused for mapping against other csv files that have Eastings and Northings
Coordinates_Column = "Site location coordinates"  
Work_Ref_Column = "Works reference number"
Promoter = "Promoter"

In [10]:
# Step 1: Identify and process Points and LineStrings
def process_coordinates(coord_string):
    if coord_string.startswith("Point:"):
        # Extract single point
        match = re.search(r"Point: ([\d.]+),([\d.]+)", coord_string)
        if match:
            return [(float(match.group(1)), float(match.group(2)))]
    elif coord_string.startswith("LineString:"):
        # Extract multiple points
        matches = re.findall(r"([\d.]+),([\d.]+)", coord_string)
        return [(float(x), float(y)) for x, y in matches]
    return []

# Apply the extraction function
DD['Coordinates'] = DD[Coordinates_Column].apply(process_coordinates)

# Step 2: Convert all coordinates to Latitude/Longitude
transformer = Transformer.from_crs("epsg:27700", "epsg:4326", always_xy=True)

def convert_coordinates(coords):
    return [transformer.transform(easting, northing) for easting, northing in coords]

DD['LatLon'] = DD['Coordinates'].apply(convert_coordinates)

# Step 3: Extract the first entry for map centring
first_entry = DD[Coordinates_Column].iloc[0]  # Get the first value in the column
first_coordinates = process_coordinates(first_entry)  # Process the coordinates
first_latlon = transformer.transform(first_coordinates[0][0], first_coordinates[0][1])  # Convert to lat/lon
map_centre = [first_latlon[1], first_latlon[0]]  # Latitude, Longitude

# Step 4: Plot on a Map using Folium
map_ = folium.Map(location=map_centre, zoom_start=16)  # Centre the map on the first coordinate

for _, row in DD.iterrows():
    lat_lon = row['LatLon']
    # Construct the pop-up content
    popup_content = (
        f"Works Reference Number: {row[Work_Ref_Column]}<br>"
        f"Promoter: {row[Promoter]}" #can add extra pop up content here - but reference them by name in the earlier cells
    )
    
    # Properly indent the if-else block inside the loop
    if len(lat_lon) == 1:
        # Add marker for a single point with a custom icon
        folium.Marker(
            location=[lat_lon[0][1], lat_lon[0][0]],  # Latitude, Longitude
            popup=folium.Popup(popup_content, max_width=300),
            icon=folium.Icon(icon="fa-person-digging", prefix="fa")  # Custom FontAwesome icon
        ).add_to(map_)
    else:
        # Add polyline for a LineString
        folium.PolyLine(
            locations=[(lat, lon) for lon, lat in lat_lon],
            color="blue",
            weight=2.5,
            opacity=0.8,
            popup=folium.Popup(popup_content, max_width=300)
        ).add_to(map_)

# Save the map to an HTML file
map_.save('map.html') 
map_